In [1]:
# Biblioteki wejściowe

import pandas as pd
import numpy as np
import numpy_financial as npf
import posixpath as os
import pathlib
from pathlib import Path
import datetime
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython
import math

from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/IMAGES')

In [2]:
# Dane źródłowe

t_zapisy=pd.read_csv(DATA /'SF_Zapisy_RAP.csv')
t_balance=pd.read_csv(DATA /'SF_Balance_RAP.csv')
t_zapisy_ST=pd.read_csv(DATA /'ST_Zapisy_SQL.csv')
t_zapisy_OST=pd.read_csv(DATA /'OST_Zapisy_SQL.csv')
t_zapisy_MIS=pd.read_csv(DATA /'MIS_Zapisy_SQL.csv')
t_zapisy_PL=pd.read_csv(DATA /'MIS_Konta_Analiza_2208.csv')

## Dane wejściowe

In [3]:
# Dane wejściowe modelu

# zakres dat
DATE_START=pd.to_datetime('2022-01-31')
DATE_R=DATE_START.year
DATE_END=datetime.datetime(DATE_R+5,12,31)

# parmetry finansowe
FX_EUR=4.82
DISCOUNT_RATE=9.0
INDEX=7.0
MARGIN=1.0

# zakres czasowy modelu
okres=pd.date_range(start=DATE_START,freq='M',end=DATE_END)
t_okres=pd.DataFrame({'DATE':okres})
t_okres['R']=t_okres['DATE'].dt.year
t_okres['M']=t_okres['DATE'].dt.month
t_okres['I']=t_okres.index

t_model=t_okres

## Bilans - Aktywa

### Wartości niematerialne i prawne

In [4]:
# Bilans - Aktywa - WNIP

t_model['Wartości niematerialne i prawne']=0.0

# Wartości niematerialne i prawne nie występują i nie są planowane

### Reczowe aktywa trwałe

#### Istniejące

In [5]:
temp=t_zapisy_ST[['NumerInwentarzowy','Nazwa',
       'ZbycieData', 'DataRozpoczeciaUzytkowania', 'DataLikwidacji', 
       'WartoscPoczatkowaBilansowaBOValue']].copy()

temp=temp[(temp['ZbycieData']=='1900-01-01')&(temp['ZbycieData']=='1900-01-01')]
temp=temp[['NumerInwentarzowy','Nazwa','DataRozpoczeciaUzytkowania','WartoscPoczatkowaBilansowaBOValue']].copy()

temp.rename(columns={'NumerInwentarzowy':'Numer','DataRozpoczeciaUzytkowania':'Data','WartoscPoczatkowaBilansowaBOValue':'Wartość początkowa'}, inplace=True)

In [6]:
tabela=temp[temp['Numer'].str.startswith('5')].sort_values(by='Data',ascending=False)
tabela=tabela.style.format({'Wartość początkowa':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '23_ST_500.tex',hrules=True,column_format='L{1.5cm}L{9cm}R{2cm}R{2cm}')
tabela

Numer,Nazwa,Data,Wartość początkowa
580/387,MINIKOPARKA GĄSIENICOWA,2022-09-05,0.00
521/386,KRUSZARKA CH430 SANDVICK,2022-06-09,0.00
581/375,LEASING MANIPULATOR,2021-04-05,0.00
582/374,LEASING ZAMIATARKA MASZYNA ROBÓT DROGOWYCH,2021-04-01,0.00
580/368,ŁADOWARKA HYUNDAI 770-9A,2020-12-14,0.00
521/359,PRZESIEWACZ WPB 821,2020-07-16,0.00
580/355,ŁADOWARKA CAT 980 H- LEASING,2020-04-20,0.00
521/345,"PRZESIEWACZ P112, KSD 21-6-2D",2020-04-20,0.00
521/344,"PRZESIEWACZ P109, KSD21-6-3D",2020-04-20,0.00
521/341,ZABUDOWA KRUSZARKI CH430-2,2020-04-20,0.00


In [7]:
tabela=temp[temp['Numer'].str.startswith('6')].sort_values(by='Data',ascending=False).head(10)
tabela=tabela.style.format({'Wartość początkowa':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '24_ST_600.tex',hrules=True,column_format='L{1.5cm}L{9cm}R{2cm}R{2cm}')
tabela

Numer,Nazwa,Data,Wartość początkowa
669/388,INSTALACJA FOTOWOLTAICZNA,2022-07-11,0.00
659/381,DETEKTOR METALU WM3100-X,2021-10-29,0.00
610/379,STACJA ŚN - STACJA TRANSFORMATOROWA,2021-07-21,0.00
622/377,MONITORING STAREGO WYROBISKA-OBROTOWA GŁOWICA,2021-06-01,0.00
644/348,"PRZENOŚNIK TAŚMOWY PT113, 650/30",2020-04-20,0.00
644/339,PRZENOŚNIK TAŚMOWY PT 101,2020-04-20,0.00
644/342,"PRZENOŚNIK TAŚMOWY PT 107 , 800/13",2020-04-20,0.00
644/343,"PRZENOŚNIK TAŚMOWY PT 108, 800/39",2020-04-20,0.00
644/346,"PRZENOŚNIK TAŚMOWY PT111, 800/36",2020-04-20,0.00
644/347,"PRZENOŚNIK TAŚMOWY PT110, 650/30",2020-04-20,0.00


In [8]:
# Zakres składników bilansowych

zakres=['A2','A3','A4','A5','A6','A7']

# Pobiera dane z wykonania
temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=-temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Rzeczowe aktywa trwałe (istniejące)'},inplace=True)
temp['R']=2022

# Aktualizacja modelu
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model['Rzeczowe aktywa trwałe (istniejące)'].fillna(0,inplace=True)


# Amortyzacja wykonana

zakres=['R5']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'Amortyzacja (istniejące)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

# Kalkulacja planu amortyzacji

t_zapisy_ST['BO']=t_zapisy_ST['WartoscPoczatkowaBilansowaBOValue']-t_zapisy_ST['OdpisyBilansoweBOValue']
t_ST=t_zapisy_ST[['ID', 'NumerInwentarzowy', 'Nazwa','BO']]
t_zapisy_OST['Obroty']=t_zapisy_OST.apply(lambda x:x['WartoscBilansowaValue'] if x['Kierunek']==1 else -x['WartoscBilansowaValue'],axis=1)

t_ST_Odpisy=t_zapisy_OST.groupby(['SrodekTrwaly'],as_index=False)[['Obroty']].sum()
t_ST=pd.merge(t_ST,t_ST_Odpisy,how='inner',left_on=['ID'],right_on=['SrodekTrwaly'])
t_ST['WNA']=t_ST['BO']+t_ST['Obroty']
t_ST=t_ST[t_ST['WNA']>1]
t_ST=t_ST[['ID', 'NumerInwentarzowy', 'Nazwa','WNA']]
t_ST_Data_Max=t_zapisy_OST.groupby(['SrodekTrwaly'],as_index=False)[['Data']].max()
t_ST_Amortyzacja=t_zapisy_OST[t_zapisy_OST['Typ']==3]
t_ST_Amortyzacja=t_ST_Amortyzacja[['SrodekTrwaly','Data','Obroty']]
t_ST_Amortyzacja=pd.merge(t_ST_Data_Max,t_ST_Amortyzacja,how='inner',on=['SrodekTrwaly','Data'])
t_ST_Amortyzacja=t_ST_Amortyzacja[t_ST_Amortyzacja['Data']=='2022-09-30']
t_ST_Amortyzacja.rename(columns={'SrodekTrwaly':'ID'},inplace=True)
t_ST=pd.merge(t_ST,t_ST_Amortyzacja,how='left',on='ID')
t_ST.rename(columns={'NumerInwentarzowy':'Numer ST', 'Nazwa':'Nazwa ST', 'WNA':'Wartość Bilansowa',  'Obroty':'Amortyzacja'},inplace=True)

t_ST['Data']=t_ST.apply(lambda x:pd.to_datetime('2022-09-30') if x['ID']==306 else x['Data'],axis=1)
t_ST['Amortyzacja']=t_ST.apply(lambda x:-x['Wartość Bilansowa']/60 if x['ID']==306 else x['Amortyzacja'],axis=1)

t_ST_Plan=pd.DataFrame()

for index,row in t_ST.iterrows():
    NUMER=row['Numer ST']
    NAZWA=row['Nazwa ST']
    WNA=row['Wartość Bilansowa']
    START=pd.to_datetime('2022-09-30')
    AM=row['Amortyzacja']

    OKRES=math.floor(abs(WNA/AM))
    RESZTA=WNA+OKRES*AM

    temp_okres=pd.date_range(start=START,freq='M',periods=OKRES+1)
    t_ST_temp=pd.DataFrame({'DATE':temp_okres})
    KONIEC=t_ST_temp['DATE'].max()

    t_ST_temp['Numer ST']=NUMER
    t_ST_temp['Nazwa ST']=NAZWA
    t_ST_temp['BO']=t_ST_temp.apply(lambda x:WNA if x['DATE']==START else 0,axis=1)
    t_ST_temp['Amortyzacja']=t_ST_temp.apply(lambda x:AM if x['DATE']>START else 0,axis=1)
    t_ST_temp['Amortyzacja']=t_ST_temp.apply(lambda x:AM-RESZTA if x['DATE']==KONIEC else x['Amortyzacja'],axis=1)
    t_ST_temp['BZ']=t_ST_temp['BO']+t_ST_temp['Amortyzacja']
    t_ST_temp['BZ']=t_ST_temp['BZ'].cumsum()


    t_ST_Plan=pd.concat([t_ST_Plan,t_ST_temp],ignore_index=True)

temp=t_ST_Plan.groupby(['DATE'],as_index=False)[['Amortyzacja']].sum()
temp.rename(columns={'Amortyzacja':'Temp'},inplace=True)

# Aktualizacja modelu

t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)
t_model['Amortyzacja (istniejące)']=t_model['Amortyzacja (istniejące)']+t_model['Temp']
t_model.drop(['Temp'],axis=1,inplace=True)
t_model['Rzeczowe aktywa trwałe (istniejące)']=t_model.apply(lambda x:x['Rzeczowe aktywa trwałe (istniejące)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['Amortyzacja (istniejące)'],axis=1)

In [9]:
temp=t_ST_Plan[t_ST_Plan['DATE']=='2022-09-30'].copy()
temp=temp[['Numer ST', 'Nazwa ST', 'BZ']].sort_values(by='BZ',ascending=False).head(10)
temp.loc['RAZEM']=temp.sum(numeric_only=True,axis=0)
temp.loc['RAZEM','Numer ST']='RAZEM'
last_row = pd.IndexSlice[temp.index[temp.index == "RAZEM"], :]

tabela=temp


tabela=tabela.style.format({'BZ':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '25_Top_10_RAT.tex',hrules=True,column_format='L{1.5cm}L{12cm}R{2cm}')
tabela

Numer ST,Nazwa ST,BZ
220/289,DROGA TECHNOLOGICZNA,"1,068,518.48"
580/355,ŁADOWARKA CAT 980 H- LEASING,"784,100.08"
644/299,PRZENOŚNIK TAŚMOWY WYRZUTOWY PT 26,"674,540.70"
610/353,ZASILANIE I STEROWANIE ZAKŁADU PRZERÓBCZEGO,"666,907.91"
200/293,FUNDAMENT I MUR OPOROWY WĘZŁA WSTĘPNEGO,"611,912.07"
521/335,KRUSZARKA CH430 SANDVICK LEASING,"492,234.28"
521/344,"PRZESIEWACZ P109, KSD21-6-3D","483,822.01"
644/304,PRZENOŚNIK TAŚMOWY PT30,"452,055.67"
521/345,"PRZESIEWACZ P112, KSD 21-6-2D","425,353.57"
581/375,LEASING MANIPULATOR,"401,769.11"


In [349]:
# Raport

temp=t_model[['R','Wartości niematerialne i prawne','Rzeczowe aktywa trwałe (istniejące)','Amortyzacja (istniejące)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Wartości niematerialne i prawne','Rzeczowe aktywa trwałe (istniejące)','Amortyzacja (istniejące)']].sum()
temp['Wartości niematerialne i prawne']=temp['Wartości niematerialne i prawne'].cumsum()
temp['Rzeczowe aktywa trwałe (istniejące)']=temp['Rzeczowe aktywa trwałe (istniejące)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '1_Aktywa_Istniejące.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela


POZYCJA,2022,2023,2024,2025,2026,2027
Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00,0.00
Rzeczowe aktywa trwałe (istniejące),"12,247.17","10,713.77","9,273.25","7,927.03","6,793.47","5,897.01"
Amortyzacja (istniejące),"-1,563.06","-1,533.40","-1,440.52","-1,346.21","-1,133.57",-896.45


In [12]:
t_ST_Plan[t_ST_Plan['Numer ST']=='580/368']

,DATE,Numer ST,Nazwa ST,BO,Amortyzacja,BZ
11147,2022-09-30,580/368,ŁADOWARKA HYUNDAI 770-9A,"29,105.29",0.00,"29,105.29"
11148,2022-10-31,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"28,359.00"
11149,2022-11-30,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"27,612.71"
11150,2022-12-31,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"26,866.42"
11151,2023-01-31,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"26,120.13"
11152,2023-02-28,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"25,373.84"
11153,2023-03-31,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"24,627.55"
11154,2023-04-30,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"23,881.26"
11155,2023-05-31,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"23,134.97"
11156,2023-06-30,580/368,ŁADOWARKA HYUNDAI 770-9A,0.00,-746.29,"22,388.68"


#### Planowane nakłady inwestycyjne

In [350]:
# Nakłady Inwestycyjne

l_ni=[
        {'Nazwa':                   'Fotowoltaika',
        'Cena [PLN]':               500000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      5*12,
        'Udział własny':        10,
        'Okres finansowania': 5*12,
        'Wartość rezydualna':   10},

        {'Nazwa':                   'Fotowoltaika - kabel',
        'Cena [PLN]':               100000,
        'Data OT':                  '2023-06-30',
        'Amortyzacja [okres]':      5*12,
        'Udział własny':        100,
        'Okres finansowania': 5*12,
        'Wartość rezydualna':   10}

]

t_ni=pd.DataFrame(l_ni)

t_tabela=t_ni.copy()
t_tabela=t_tabela[['Nazwa', 'Cena [PLN]', 'Data OT','Udział własny','Okres finansowania','Wartość rezydualna']]
t_tabela.loc['RAZEM','Cena [PLN]']=t_tabela['Cena [PLN]'].sum()
t_tabela.loc['RAZEM','Nazwa']='RAZEM'
t_tabela_fv=t_tabela.style.format({'Cena [PLN]':'{0:,.2f}','Udział własny':'{0:,.2f}','Okres finansowania':'{0:,.2f}','Wartość rezydualna':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv.to_latex(ASSETS / '2_Nakłady_Inwestycyjne.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}R{2cm}R{2cm}R{1.5cm}')
t_tabela_fv


Nazwa,Cena [PLN],Data OT,Udział własny,Okres finansowania,Wartość rezydualna
Fotowoltaika,"500,000.00",2023-06-30,10.00,60.00,10.00
Fotowoltaika - kabel,"100,000.00",2023-06-30,100.00,60.00,10.00
RAZEM,"600,000.00",nan,nan,nan,nan


#### Rozliczenie nakładów inwestycyjnych

In [351]:
# Rozliczenie inwestycji

t_capex=pd.DataFrame()

for index,row in t_ni.iterrows():

    NAME=row['Nazwa']
    CENA=row['Cena [PLN]']
    DATA_OT=pd.to_datetime(row['Data OT'])
    AM=row['Amortyzacja [okres]']
    DP=row['Udział własny']
    T=row['Okres finansowania']
    RV=row['Wartość rezydualna']

    okres=pd.date_range(start=DATA_OT,freq='M',periods=AM+1)
    t_capex_temp=pd.DataFrame({'DATE':okres})
    t_capex_temp['R']=t_capex_temp['DATE'].dt.year
    t_capex_temp['M']=t_capex_temp['DATE'].dt.month
    t_capex_temp['I']=t_capex_temp.index
    t_capex_temp['Nazwa']=NAME
    t_capex_temp['Zakup']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT else CENA, axis=1)
    t_capex_temp['Amortyzacja']=t_capex_temp.apply(lambda x:0 if x['I']==0 else CENA/AM, axis=1)
    t_capex_temp['Aktywo']=t_capex_temp['Zakup'].cumsum()-t_capex_temp['Amortyzacja'].cumsum()
    t_capex_temp['Finansowanie własne']=t_capex_temp.apply(lambda x:CENA*DP/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['Finansowanie obce']=t_capex_temp.apply(lambda x:CENA*(100-DP)/100 if x['I']==0 else 0, axis=1)
    t_capex_temp['PPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ppmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)
    t_capex_temp['IPMT']=t_capex_temp.apply(lambda x:0 if (x['I']==0 or x['I']>T) else -npf.ipmt((INDEX+MARGIN)/1200,x['I'],T,CENA*(100-DP)/100,-CENA*(RV)/100),axis=1)
    t_capex_temp['RV']=t_capex_temp.apply(lambda x:CENA*(RV)/100 if x['DATE']==DATA_OT+pd.DateOffset(months=T) else 0, axis=1)
    t_capex_temp['WNA']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT+pd.DateOffset(months=T) else x['Aktywo'], axis=1)

    t_capex=pd.concat([t_capex,t_capex_temp],ignore_index=True)

t_capex_model=t_capex.groupby(['DATE','R'],as_index=False)[['Zakup', 'Amortyzacja','Finansowanie własne', 'Finansowanie obce', 'PPMT', 'IPMT', 'RV']].sum()

#### Aktualizacja modelu

In [352]:
# Aktualizacja modelu o nakłady inwestycyjne i ich finansowanie

t_capex_model['Aktywo']=t_capex_model['Zakup']-t_capex_model['Amortyzacja']
t_capex_model['Pasywo']=t_capex_model['Finansowanie obce']-t_capex_model['PPMT']-t_capex_model['RV']

temp=t_capex_model[['DATE']].copy()
temp['Nakłady inwestycyjne']=-t_capex_model['Zakup']
temp['Kredyty bankowe i leasingi (inwestycje)']=t_capex_model['Finansowanie obce']
temp['Amortyzacja (inwestycje)']=-t_capex_model['Amortyzacja']
temp['Koszty finansowe (inwestycje)']=-t_capex_model['IPMT']
temp['Spłata kredytów i leasingów (inwestycje)']=-t_capex_model['PPMT']-t_capex_model['RV']
temp['Rzeczowe aktywa trwałe (inwestycje)']=t_capex_model['Aktywo']
temp['Zobowiązania długoterminowe (inwestycje)']=t_capex_model['Pasywo']
t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

In [353]:
# Raport

temp=t_model[['R','Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Nakłady inwestycyjne','Rzeczowe aktywa trwałe (inwestycje)','Amortyzacja (inwestycje)','Kredyty bankowe i leasingi (inwestycje)','Zobowiązania długoterminowe (inwestycje)','Spłata kredytów i leasingów (inwestycje)','Koszty finansowe (inwestycje)']].sum()

temp['Rzeczowe aktywa trwałe (inwestycje)']=temp['Rzeczowe aktywa trwałe (inwestycje)'].cumsum()
temp['Zobowiązania długoterminowe (inwestycje)']=temp['Zobowiązania długoterminowe (inwestycje)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '4_Wpływ_Inwestycji_SF.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Nakłady inwestycyjne,0.00,-600.00,0.00,0.00,0.00,0.00
Rzeczowe aktywa trwałe (inwestycje),0.00,540.00,420.00,300.00,180.00,60.00
Amortyzacja (inwestycje),0.00,-60.00,-120.00,-120.00,-120.00,-120.00
Kredyty bankowe i leasingi (inwestycje),0.00,450.00,0.00,0.00,0.00,0.00
Zobowiązania długoterminowe (inwestycje),0.00,417.62,348.85,274.37,193.71,106.36
Spłata kredytów i leasingów (inwestycje),0.00,-32.38,-68.77,-74.48,-80.66,-87.35
Koszty finansowe (inwestycje),0.00,-17.46,-30.92,-25.22,-19.03,-12.34


### Projekty rozwojowe

#### Istniejące

In [354]:
# Długoterminowe rozliczenia międzyokresowe - wykonanie

zakres=['A11','A19','A10']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=-temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'AT_DRM_W'},inplace=True)
temp['R']=2022

t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model['AT_DRM_W'].fillna(0,inplace=True)
temp=t_zapisy[(t_zapisy['N_Dokument'].str.contains('RMK'))&(t_zapisy['Konto'].str.startswith('409'))].copy()
temp=temp.groupby(['M','R'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'PL_KO_RMK_W'},inplace=True)
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

WART=t_model['AT_DRM_W'].sum()
RMK=-t_model['PL_KO_RMK_W'].iloc[8]
TENOR=int(WART/RMK)

t_model['Długoterminowe rozliczenia']=t_model.apply(lambda x:x['AT_DRM_W'] if x['DATE']<=pd.to_datetime('2022-09-30') else -RMK,axis=1)
t_model['Koszty zrealizowanych projektów (istniejące)']=t_model.apply(lambda x:x['PL_KO_RMK_W'] if x['DATE']<pd.to_datetime('2022-09-30') else -RMK,axis=1)
t_model.drop(['AT_DRM_W','PL_KO_RMK_W'],axis=1,inplace=True)

In [355]:
# Raport

temp=t_model[['R','Długoterminowe rozliczenia','Koszty zrealizowanych projektów (istniejące)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Długoterminowe rozliczenia','Koszty zrealizowanych projektów (istniejące)']].sum()
temp['Długoterminowe rozliczenia']=temp['Długoterminowe rozliczenia'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '5_Istniejące_RMK.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Długoterminowe rozliczenia,"5,742.99","5,283.73","4,824.47","4,365.22","3,905.96","3,446.70"
Koszty zrealizowanych projektów (istniejące),-383.78,-459.26,-459.26,-459.26,-459.26,-459.26


#### Planowane

In [356]:
# Planowane nakłady na rozwój złoza

# Nakłady obejmują:
# Odszkodowanie za przedwczesny wyrąb
# Prace geodezyjne
# Wyrąb
# Załadunek i przemieszczenie nadkłądu do starego wyrobiska

l_ni=[

        {'Nazwa':                   'Etap południowy XI',
        'Powierzchnia [mkw]':       20000,
        'Cena [PLN]':               1600000,
        'Data odkrycia':           '2024-03-31',
        'Finansowanie':             'Środki własne'},

        {'Nazwa':                   'Etap południowy XII',
        'Powierzchnia [mkw]':       20000,
        'Cena [PLN]':               1800000,
        'Data odkrycia':           '2025-03-31',
        'Finansowanie':             'Środki własne'}
]

t_ni=pd.DataFrame(l_ni)

t_tabela=t_ni.copy()
t_tabela.loc['RAZEM','Cena [PLN]']=t_tabela['Cena [PLN]'].sum()
t_tabela.loc['RAZEM','Powierzchnia [mkw]']=t_tabela['Powierzchnia [mkw]'].sum()
t_tabela.loc['RAZEM','Nazwa']='RAZEM'
t_tabela.fillna('',inplace=True)
t_tabela_fv=t_tabela.style.format({'Cena [PLN]':'{0:,.2f}','Powierzchnia [mkw]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv.to_latex(ASSETS / '6_Nakłady_Projekty.tex',hrules=True,column_format='L{4cm}R{2cm}R{2cm}R{2cm}R{4cm}')
display(t_tabela_fv)


Nazwa,Powierzchnia [mkw],Cena [PLN],Data odkrycia,Finansowanie
Etap południowy XI,"20,000.00","1,600,000.00",2024-03-31,Środki własne
Etap południowy XII,"20,000.00","1,800,000.00",2025-03-31,Środki własne
RAZEM,"40,000.00","3,400,000.00",,


#### Rozliczenie

In [357]:
# Rozliczenie projektów

LASY_DZIERZAWA=0.47
LASY_WYRAB=0.24
DATA_KONCESJA=datetime.datetime(2050,8,24)

t_capex=pd.DataFrame()

for index,row in t_ni.iterrows():

    NAME=row['Nazwa']
    CENA=row['Cena [PLN]']
    DATA_OT=pd.to_datetime(row['Data odkrycia'])
    AM=int((DATA_KONCESJA-DATA_OT)/np.timedelta64(1,'M'))
    LASY_D=row['Powierzchnia [mkw]']*LASY_DZIERZAWA
    LASY_W=row['Powierzchnia [mkw]']*LASY_WYRAB

    okres=pd.date_range(start=DATA_OT,freq='M',periods=AM+1)
    t_capex_temp=pd.DataFrame({'DATE':okres})
    t_capex_temp['R']=t_capex_temp['DATE'].dt.year
    t_capex_temp['M']=t_capex_temp['DATE'].dt.month
    t_capex_temp['I']=t_capex_temp.index
    t_capex_temp['Nazwa']=NAME
    t_capex_temp['Zakup']=t_capex_temp.apply(lambda x:0 if x['DATE']!=DATA_OT else CENA, axis=1)
    t_capex_temp['Amortyzacja']=t_capex_temp.apply(lambda x:0 if x['I']==0 else CENA/AM, axis=1)
    t_capex_temp['Aktywo']=t_capex_temp['Zakup']-t_capex_temp['Amortyzacja']
    t_capex_temp['Lasy']=t_capex_temp.apply(lambda x:0 if (x['I']==0) else LASY_D+LASY_W,axis=1)


    t_capex=pd.concat([t_capex,t_capex_temp],ignore_index=True)


t_capex_model=t_capex.groupby(['DATE','R'],as_index=False)[['Zakup', 'Amortyzacja','Lasy']].sum()
t_capex_model['Aktywo']=t_capex_model['Zakup']-t_capex_model['Amortyzacja']
temp=t_capex_model[['DATE']].copy()
temp['Nakłady inwestycyjne (rozwój złoża)']=-t_capex_model['Zakup']
temp['Koszty zrealizowanych projektów (planowane)']=-t_capex_model['Amortyzacja']
temp['Podatki i opłaty (planowane) - Lasy Państwowe']=-t_capex_model['Lasy']
temp['Długoterminowe rozliczenia (planowane)']=t_capex_model['Aktywo']

t_model=pd.merge(t_model,temp,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

In [358]:
# Raport

temp=t_model[['R','Nakłady inwestycyjne (rozwój złoża)','Długoterminowe rozliczenia (planowane)','Koszty zrealizowanych projektów (planowane)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Nakłady inwestycyjne (rozwój złoża)','Długoterminowe rozliczenia (planowane)','Koszty zrealizowanych projektów (planowane)']].sum()

temp['Długoterminowe rozliczenia (planowane)']=temp['Długoterminowe rozliczenia (planowane)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '7_Planowane_RMK.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Nakłady inwestycyjne (rozwój złoża),0.00,0.00,"-1,600.00","-1,800.00",0.00,0.00
Długoterminowe rozliczenia (planowane),0.00,0.00,"1,554.43","3,240.38","3,108.57","2,976.76"
Koszty zrealizowanych projektów (planowane),0.00,0.00,-45.57,-114.05,-131.81,-131.81


In [359]:
# Zapasy

zakres=['A12','A13']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=-temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zapasy'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [360]:
# Należności

zakres=['A14','A15','A16','A17']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=-temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Należności'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [361]:
# Środki pieniężne

zakres=['A18']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=-temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Środki pieniężne'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

### Kapitały własne

In [362]:
# Kapitał podstawowy

zakres=['P1']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Kapitał podstawowy'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Kapitał podstawowy']
t_model['Kapitał podstawowy']=t_model.apply(lambda x:x['Kapitał podstawowy'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)


In [363]:
# Kapitał zapasowy

zakres=['P2']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Kapitał zapasowy'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Kapitał zapasowy']
t_model['Kapitał zapasowy']=t_model.apply(lambda x:x['Kapitał zapasowy'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)

In [364]:
# Kapitał Akt

zakres=['P3']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Kapitał z aktualizacji wyceny'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Kapitał z aktualizacji wyceny']
t_model['Kapitał z aktualizacji wyceny']=t_model.apply(lambda x:x['Kapitał z aktualizacji wyceny'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)

In [365]:
# Kapitał rezerwowy

zakres=['P4']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Pozostałe kapitały rezerwowe'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Pozostałe kapitały rezerwowe']
t_model['Pozostałe kapitały rezerwowe']=t_model.apply(lambda x:x['Pozostałe kapitały rezerwowe'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)

In [366]:
# Rozliczenie zysku

zakres=['P5']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zysk (strata) z lat ubiegłych'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Zysk (strata) z lat ubiegłych']
t_model['Zysk (strata) z lat ubiegłych']=t_model.apply(lambda x:x['Zysk (strata) z lat ubiegłych'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)

In [367]:
# Dywidenda

zakres=['P6']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Odpisy w ciągu roku'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

LAST=t_model.loc[8,'Odpisy w ciągu roku']
t_model['Odpisy w ciągu roku']=t_model.apply(lambda x:x['Odpisy w ciągu roku'] if x['DATE']<=pd.to_datetime('2022-09-30') else LAST,axis=1)

In [368]:
# Raport

temp=t_model[['R','Kapitał podstawowy','Kapitał zapasowy','Kapitał z aktualizacji wyceny','Pozostałe kapitały rezerwowe','Zysk (strata) z lat ubiegłych','Odpisy w ciągu roku']].copy()
temp=temp.groupby(['R'],as_index=False)[['Kapitał podstawowy','Kapitał zapasowy','Kapitał z aktualizacji wyceny','Pozostałe kapitały rezerwowe','Zysk (strata) z lat ubiegłych','Odpisy w ciągu roku']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '8_Kapitały_092022.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Kapitał podstawowy,"1,240.00",0.00,0.00,0.00,0.00,0.00
Kapitał zapasowy,"10,983.60",0.00,0.00,0.00,0.00,0.00
Kapitał z aktualizacji wyceny,74.06,0.00,0.00,0.00,0.00,0.00
Pozostałe kapitały rezerwowe,"4,653.93",0.00,0.00,0.00,0.00,0.00
Zysk (strata) z lat ubiegłych,0.00,0.00,0.00,0.00,0.00,0.00
Odpisy w ciągu roku,"-3,000.00",0.00,0.00,0.00,0.00,0.00


### Zobowiązania i rezerwy na zobowiązania

In [369]:
# Rezerwy na zobowiązania

zakres=['P7','P8','P9']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Rezerwy na zobowiązania'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [370]:
# Kredyt inwestycyjny PEKAO

zakres=['135-002','134-003']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL','Kwota_CR']].sum()

temp['Kwota_CR']=-temp['Kwota_CR']
temp.rename(columns={'Kwota_SAL':'Zobowiązania długoterminowe (PEKAO)','Kwota_CR':'Spłaty kredytów i leasingów (PEKAO)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

PEKAO_OUT=t_model['Zobowiązania długoterminowe (PEKAO)'].sum()
PEKAO_KAP=-t_model.loc[8,'Spłaty kredytów i leasingów (PEKAO)']
PEKAO_TEN=int(PEKAO_OUT/PEKAO_KAP)
PEKAO_REST=PEKAO_OUT-PEKAO_KAP*(PEKAO_TEN+1)
PEKAO_MATURITY=datetime.datetime(2022,9,30)+relativedelta(months=+(PEKAO_TEN))+relativedelta(day=31)
PEKAO_MATURITY
t_model['CF_MIN_KR_PEKAO_B']=t_model.apply(lambda x:-PEKAO_KAP if (x['DATE']>pd.to_datetime('2022-09-30') and x['DATE']<PEKAO_MATURITY) else 0,axis=1)
t_model['CF_MIN_KR_PEKAO_B']=t_model.apply(lambda x:-(PEKAO_KAP+PEKAO_REST) if x['DATE']==PEKAO_MATURITY else x['CF_MIN_KR_PEKAO_B'],axis=1)


t_model['Spłaty kredytów i leasingów (PEKAO)']=t_model.apply(lambda x:x['Spłaty kredytów i leasingów (PEKAO)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['CF_MIN_KR_PEKAO_B'],axis=1)
t_model['Zobowiązania długoterminowe (PEKAO)']=t_model.apply(lambda x:x['Zobowiązania długoterminowe (PEKAO)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['Spłaty kredytów i leasingów (PEKAO)'],axis=1)

t_model.drop(['CF_MIN_KR_PEKAO_B'],axis=1,inplace=True)


In [371]:
#  TARCZA

zakres=['241-02','240-03']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL','Kwota_CR']].sum()

temp['Kwota_CR']=-temp['Kwota_CR']
temp.rename(columns={'Kwota_SAL':'Zobowiązania długoterminowe (Tarcza)','Kwota_CR':'Spłaty kredytów i leasingów (Tarcza)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]

t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

TARCZA_OUT=t_model['Zobowiązania długoterminowe (Tarcza)'].sum()
TARCZA_KAP=-t_model.loc[8,'Spłaty kredytów i leasingów (Tarcza)']
TARCZA_TEN=int(TARCZA_OUT/TARCZA_KAP)
TARCZA_REST=TARCZA_OUT-TARCZA_KAP*(TARCZA_TEN+1)
TARCZA_MATURITY=datetime.datetime(2022,9,30)+relativedelta(months=+(TARCZA_TEN))+relativedelta(day=31)
TARCZA_MATURITY

t_model['CF_MIN_KR_TARCZA_B']=t_model.apply(lambda x:-TARCZA_KAP if (x['DATE']>pd.to_datetime('2022-09-30') and x['DATE']<TARCZA_MATURITY) else 0,axis=1)
t_model['CF_MIN_KR_TARCZA_B']=t_model.apply(lambda x:-(TARCZA_KAP+TARCZA_REST) if x['DATE']==TARCZA_MATURITY else x['CF_MIN_KR_TARCZA_B'],axis=1)

t_model['Spłaty kredytów i leasingów (Tarcza)']=t_model.apply(lambda x:x['Spłaty kredytów i leasingów (Tarcza)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['CF_MIN_KR_TARCZA_B'],axis=1)
t_model['Zobowiązania długoterminowe (Tarcza)']=t_model.apply(lambda x:x['Zobowiązania długoterminowe (Tarcza)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['Spłaty kredytów i leasingów (Tarcza)'],axis=1)

t_model.drop(['CF_MIN_KR_TARCZA_B'],axis=1,inplace=True)


In [372]:
# Kredyt obrotowy


zakres=['134-001']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Kredyt obrotowy (PEKAO)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [373]:
# Zobowiązania z tytułu leasingów

zakres=['249-301-03','249-301-04','249-301-05','249-301-06','249-301-07','249-301-08','249-301-09','249-301-10','249-301-11','249-301-12','249-301-13','249-301-14']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zobowiązania długoterminowe (leasing)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)


# Zakładam rozliczenie według ostatniej raty kapitałowej do pełnej spłaty
# Jeśli jest RV zakładam, ze będziemy wydłuać leasing

zakres=['249-301-03','249-301-05','249-301-06','249-301-07','249-301-08','249-301-09','249-301-10','249-301-11','249-301-12','249-301-13','249-301-14']
t_leas_fv=pd.DataFrame()

for konto in zakres:

    temp=t_balance[t_balance['Konto']==konto]
    temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
    L_OUT=temp['Kwota_SAL'].sum()
    L_KAP=-temp['Kwota_SAL'].iloc[-1]
    L_TEN=int(L_OUT/L_KAP)
    okres=pd.date_range(start=pd.to_datetime('2022-10-31'),freq='M',periods=L_TEN)
    t_leas=pd.DataFrame({'DATE':okres})
    t_leas['Konto']=konto
    t_leas['Spłaty kredytów i leasingów (leasing)']=-L_KAP
    t_leas_fv=pd.concat([t_leas_fv,t_leas],ignore_index=True)

t_leas_fv=t_leas_fv.groupby(['DATE'],as_index=False)[['Spłaty kredytów i leasingów (leasing)']].sum()
t_model=pd.merge(t_model,t_leas_fv,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

t_model['Zobowiązania długoterminowe (leasing)']=t_model.apply(lambda x:x['Zobowiązania długoterminowe (leasing)'] if x['DATE']<=pd.to_datetime('2022-09-30') else x['Spłaty kredytów i leasingów (leasing)'], axis=1)

In [374]:
# Raport

temp=t_model[['R','Rezerwy na zobowiązania','Zobowiązania długoterminowe (PEKAO)','Spłaty kredytów i leasingów (PEKAO)','Zobowiązania długoterminowe (Tarcza)','Spłaty kredytów i leasingów (Tarcza)','Zobowiązania długoterminowe (leasing)','Spłaty kredytów i leasingów (leasing)']].copy()
temp=temp.groupby(['R'],as_index=False)[['Rezerwy na zobowiązania','Zobowiązania długoterminowe (PEKAO)','Spłaty kredytów i leasingów (PEKAO)','Zobowiązania długoterminowe (Tarcza)','Spłaty kredytów i leasingów (Tarcza)','Zobowiązania długoterminowe (leasing)','Spłaty kredytów i leasingów (leasing)']].sum()

temp['Rezerwy na zobowiązania']=temp['Rezerwy na zobowiązania'].cumsum()
temp['Zobowiązania długoterminowe (PEKAO)']=temp['Zobowiązania długoterminowe (PEKAO)'].cumsum()
temp['Zobowiązania długoterminowe (Tarcza)']=temp['Zobowiązania długoterminowe (Tarcza)'].cumsum()
temp['Zobowiązania długoterminowe (leasing)']=temp['Zobowiązania długoterminowe (leasing)'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '9_Rezerwy_Zobowiązania_DT.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Rezerwy na zobowiązania,"2,049.32","2,049.32","2,049.32","2,049.32","2,049.32","2,049.32"
Zobowiązania długoterminowe (PEKAO),470.00,39.17,39.17,39.17,39.17,39.17
Spłaty kredytów i leasingów (PEKAO),-470.00,-430.83,0.00,0.00,0.00,0.00
Zobowiązania długoterminowe (Tarcza),223.07,37.18,37.18,37.18,37.18,37.18
Spłaty kredytów i leasingów (Tarcza),-446.14,-185.89,0.00,0.00,0.00,0.00
Zobowiązania długoterminowe (leasing),"1,323.19",649.09,341.83,274.96,250.87,226.78
Spłaty kredytów i leasingów (leasing),-171.36,-674.10,-307.26,-66.87,-24.09,-24.09


### Pozostałe zobowiązania

In [375]:
# Zobowiązania z tytułu dostaw i usług

zakres=['P14']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zobowiązania z tytułu dostaw i usług'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)


In [376]:
# Zobowiązania z tytułu podatków - wynagrodzeń

zakres=['P16','P15']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zobowiązania krótkoterminowe (podatki i wynagrodzenia)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)


In [377]:
# Pozostałe zobowiązania inne niz leasing i urzędy

zakres=['249-100-01','249-100-03','249-100-02','249-100-09','249-100-11','249-100-12','249-200-01','249-201-01','249-201-02','249-201-03','249-300-01','249-300-02','249-100-04']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Zobowiązania krótkoterminowe (pozostałe)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [378]:
#  Rozliczenie dywidendy

zakres=['249-400-02']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'Zobowiązania krótkoterminowe (dywidenda)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

KSCCH_TEN=4
KSCCH_MATURITY=datetime.datetime(2022,9,30)+relativedelta(months=+(KSCCH_TEN))+relativedelta(day=31)
KSCCH_MATURITY
KSCCH_KAP=500000

t_model['Inne wydatki (dywidenda)']=t_model.apply(lambda x:-KSCCH_KAP if (x['DATE']>pd.to_datetime('2022-09-30') and x['DATE']<KSCCH_MATURITY) else 0,axis=1)
t_model['Zobowiązania krótkoterminowe (dywidenda)']=t_model['Inne wydatki (dywidenda)']+t_model['Zobowiązania krótkoterminowe (dywidenda)']


In [379]:
# ZFSS

zakres=['850-000']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Fundusz Socjalny'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)


In [380]:
# Fundusz Likwidacji ZG

zakres=['851-001']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Fundusz Likwidacji Zakładu Górniczego'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [381]:
# Rozliczenia międzyokresowe przychodów

zakres=['840-100']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Rozliczenia międzyokresowe przychodów'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [382]:
# Pożyczka WFOS

zakres=['240-01']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Pożyczka WFOŚiGW'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [383]:
# Raport

temp=t_model[['R','Zobowiązania z tytułu dostaw i usług','Zobowiązania krótkoterminowe (podatki i wynagrodzenia)','Zobowiązania krótkoterminowe (pozostałe)','Zobowiązania krótkoterminowe (dywidenda)','Fundusz Socjalny','Fundusz Likwidacji Zakładu Górniczego','Pożyczka WFOŚiGW']].copy()
temp=temp.groupby(['R'],as_index=False)[['Zobowiązania z tytułu dostaw i usług','Zobowiązania krótkoterminowe (podatki i wynagrodzenia)','Zobowiązania krótkoterminowe (pozostałe)','Zobowiązania krótkoterminowe (dywidenda)','Fundusz Socjalny','Fundusz Likwidacji Zakładu Górniczego','Pożyczka WFOŚiGW']].sum()

temp['Zobowiązania z tytułu dostaw i usług']=temp['Zobowiązania z tytułu dostaw i usług'].cumsum()
temp['Zobowiązania krótkoterminowe (podatki i wynagrodzenia)']=temp['Zobowiązania krótkoterminowe (podatki i wynagrodzenia)'].cumsum()
temp['Zobowiązania krótkoterminowe (pozostałe)']=temp['Zobowiązania krótkoterminowe (pozostałe)'].cumsum()
temp['Zobowiązania krótkoterminowe (dywidenda)']=temp['Zobowiązania krótkoterminowe (dywidenda)'].cumsum()
temp['Fundusz Socjalny']=temp['Fundusz Socjalny'].cumsum()
temp['Fundusz Likwidacji Zakładu Górniczego']=temp['Fundusz Likwidacji Zakładu Górniczego'].cumsum()
temp['Pożyczka WFOŚiGW']=temp['Pożyczka WFOŚiGW'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()
tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '10_Pozostałe_Zobowiązania_Niehandlowe.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Zobowiązania z tytułu dostaw i usług,"5,535.75","5,535.75","5,535.75","5,535.75","5,535.75","5,535.75"
Zobowiązania krótkoterminowe (podatki i wynagrodzenia),311.46,311.46,311.46,311.46,311.46,311.46
Zobowiązania krótkoterminowe (pozostałe),462.14,462.14,462.14,462.14,462.14,462.14
Zobowiązania krótkoterminowe (dywidenda),0.00,0.00,0.00,0.00,0.00,0.00
Fundusz Socjalny,62.32,62.32,62.32,62.32,62.32,62.32
Fundusz Likwidacji Zakładu Górniczego,839.45,839.45,839.45,839.45,839.45,839.45
Pożyczka WFOŚiGW,0.00,0.00,0.00,0.00,0.00,0.00


## Rachunek Wyników

### Przychody

In [384]:
# Przychody netto ze sprzedazy produktów 

zakres=['700-100','700-300','703-100']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'Przychody ze sprzedaży produktów (zrealizowane)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [385]:
# Plan przychodów

# Dla okresu 2022 + 5 lat = 6 lat
# w - wektor

w_lata=np.array([2022,2023,2024,2025,2026,2027]).reshape(6,1)
w_produkty=np.array(['Grysy','Mieszanka','Piasek','Tłuczeń','Kliniec'])
w_mce=np.array([[1,3],[2,6],[3,9],[4,9],[5,9],[6,9],[7,10],[8,12],[9,12],[10,10],[11,8],[12,3]])

w_ilość=np.array([1170,850,900,950,1000,1050]).reshape(6,1)
w_struktura=np.array([[57,28,11,3,1],[56,28,10,5,1],[42,32,10,15,1],[47,27,10,15,1],[49,23,10,17,1],[50,19,10,20,1]])
w_cena=np.array([33.92,17,79,29.08,28.26])

w_PPI=np.array([1,1.07,1.05,1.025,1.02,1.01])
w_PPI=np.cumprod(w_PPI).reshape(6,1)

#--------------------------------------------------------------------------------------------------------------------
#KALKULACJA
#--------------------------------------------------------------------------------------------------------------------

w_ile=w_ilość*w_struktura/100*1000
w_ile=w_ile.sum(axis=1)
w_ile=w_ile.reshape(6,1)
w_przychody=w_ilość*w_struktura*w_cena*w_PPI/100*1000
w_przychody=w_przychody.sum(axis=1)
w_przychody=w_przychody.reshape(6,1)
w_przychody=np.concatenate([w_lata,w_ile, w_przychody],axis=1)
t_przychody=pd.DataFrame(w_przychody,columns=['R','PL_Ile_Y_P','PL_Przych_Y_P'])
t_mce=pd.DataFrame(w_mce,columns=['M','PL_Str'])
t_przychody=pd.merge(t_przychody,t_mce,how='cross')
t_przychody['PL_Ile_P']=t_przychody['PL_Ile_Y_P']*t_przychody['PL_Str']/100
t_przychody['Przychody ze sprzedaży produktów (planowane)']=t_przychody['PL_Przych_Y_P']*t_przychody['PL_Str']/100
t_przychody['DATE']=t_przychody.apply(lambda x:datetime.datetime(int(x['R']),int(x['M']),1),axis=1)+MonthEnd(0)
t_przychody=t_przychody[['DATE','PL_Ile_P','Przychody ze sprzedaży produktów (planowane)']].copy()
t_przychody['Przychody ze sprzedaży produktów (planowane)']=t_przychody.apply(lambda x:0 if x['DATE']<=pd.to_datetime('2022-09-30') else x['Przychody ze sprzedaży produktów (planowane)'],axis=1)

t_model=pd.merge(t_model,t_przychody,how='left',on=['DATE'])
t_model.fillna(0,inplace=True)

w_struktura.sum(axis=1)


array([100, 100, 100, 100, 100, 100])

In [386]:
# Raport


w_temp=np.concatenate([w_lata,w_ile, w_struktura],axis=1)
t_temp=pd.DataFrame(w_temp,columns=['Lata','Ilość','Grysy','Mieszanka','Piasek','Tłuczeń','Kliniec'])
t_temp['Lata']=t_temp['Lata'].astype(int)
tabela=t_temp

tabela=tabela.style.format({'Ilość':'{0:,.2f}','Grysy':'{0:,.2f}','Mieszanka':'{0:,.2f}','Piasek':'{0:,.2f}','Tłuczeń':'{0:,.2f}','Kliniec':'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '11.0_Przychody.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

Lata,Ilość,Grysy,Mieszanka,Piasek,Tłuczeń,Kliniec
2022,"1,170,000.00",57.00,28.00,11.00,3.00,1.00
2023,"850,000.00",56.00,28.00,10.00,5.00,1.00
2024,"900,000.00",42.00,32.00,10.00,15.00,1.00
2025,"950,000.00",47.00,27.00,10.00,15.00,1.00
2026,"1,000,000.00",49.00,23.00,10.00,17.00,1.00
2027,"1,050,000.00",50.00,19.00,10.00,20.00,1.00


In [387]:
w_struktura.sum(axis=1)

array([100, 100, 100, 100, 100, 100])

In [388]:
t_model[['DATE','PL_Ile_P']].head(12)

,DATE,PL_Ile_P
0,2022-01-31,"35,100.00"
1,2022-02-28,"70,200.00"
2,2022-03-31,"105,300.00"
3,2022-04-30,"105,300.00"
4,2022-05-31,"105,300.00"
5,2022-06-30,"105,300.00"
6,2022-07-31,"117,000.00"
7,2022-08-31,"140,400.00"
8,2022-09-30,"140,400.00"
9,2022-10-31,"117,000.00"


In [389]:
# Pozostałe przychody

zakres=['R2','R3','R4']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Przychody pozostałe'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [390]:
# Raport

temp=t_model[['R','Przychody ze sprzedaży produktów (zrealizowane)','Przychody ze sprzedaży produktów (planowane)','Przychody pozostałe']].copy()
temp=temp.groupby(['R'],as_index=False)[['Przychody ze sprzedaży produktów (zrealizowane)','Przychody ze sprzedaży produktów (planowane)','Przychody pozostałe']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '11_Przychody.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Przychody ze sprzedaży produktów (zrealizowane),"36,729.70",0.00,0.00,0.00,0.00,0.00
Przychody ze sprzedaży produktów (planowane),"8,338.91","30,369.84","32,590.38","36,186.51","39,534.16","42,588.18"
Przychody pozostałe,-216.06,0.00,0.00,0.00,0.00,0.00
RAZEM,"44,852.55","30,369.84","32,590.38","36,186.51","39,534.16","42,588.18"


### Koszty operacyjne

#### Zyżycie materiałów i energii

In [391]:
# Zuzycie materiałów produkcyjnych

zakres=['401-100-01']
NAZWA='Zużycie materiałów produkcyjnych'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)
t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([1,
    1.2,
    1.08,
    1.05,
    1.025,
    1])
w_SAV=np.array([1,0.9,1,1,1,1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=10

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------

temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.1 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)

temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------
t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])

#temp_zap
#temp


In [392]:
# Zuzycie materiałów pozostałych

zakres=['401-100-02','401-100-03','401-100-04']
NAZWA='Zuzycie materiałów pozostałych'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)
t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([1,
    1.08,
    1.08,
    1.05,
    1.025,
    1])
w_SAV=np.array([1,0.8,0.9,1,1,1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=80

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.1 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------
t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])

#temp_zap
#temp

In [393]:
# Paliwo

zakres=['401-100-05']
NAZWA='Zużycie paliwa'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)
t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.2,
    1.15,
    1.1,
    1.05,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.9,
    0.9,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=20

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.2 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)

temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------
t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])

#temp_zap
#temp

In [394]:
# Zuyżcie energii

zakres=['401-200-01']
NAZWA='Zużycie energii'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)
t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.25,
    1.1,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.85,
    0.9,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=10

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.2 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------
t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])

#temp_zap
#temp

In [395]:
# Raport

temp=t_model[['R','Zużycie materiałów produkcyjnych','Zuzycie materiałów pozostałych','Zużycie paliwa','Zużycie energii']].copy()
temp=temp.groupby(['R'],as_index=False)[['Zużycie materiałów produkcyjnych','Zuzycie materiałów pozostałych','Zużycie paliwa','Zużycie energii']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '12_Zużycie materiałów i energii.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Zużycie materiałów produkcyjnych,"-1,839.21","-1,957.53","-2,104.28","-2,199.14","-2,243.51","-2,232.91"
Zuzycie materiałów pozostałych,-26.09,-22.18,-21.58,-22.69,-23.28,-23.30
Zużycie paliwa,"-1,603.88","-1,211.97","-1,321.65","-1,527.81","-1,681.89","-1,803.58"
Zużycie energii,"-3,274.43","-2,408.41","-2,512.63","-2,772.97","-2,980.37","-3,196.42"
RAZEM,"-6,743.62","-5,600.09","-5,960.14","-6,522.61","-6,929.06","-7,256.20"


#### Usługi obce

In [396]:
# Usługi obce - Transport do klienta 

zakres=['402-100-01','402-100-06','402-100-10','402-100-20','402-101-01','402-200-02','402-200-03','402-200-04','402-200-12','402-200-10']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'Koszty bezpośrednie sprzedaży (transport)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [397]:
# Usługi obce - Waśpol

zakres=['402-200-06','402-200-07','402-200-08']
NAZWA='Usługi obce - obsługa złoża'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)
t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.15,
    1.1,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.15 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)

temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [398]:
# Usługi obce - transport kamienia na plac składowy - transport mieszanki na plac

zakres=['402-200-09']
NAZWA='Usługi obce - transport mieszanki na plac'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.15,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    0.9,
    0.8,
    0.8,
    0.8])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.1 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [399]:
# Usługi obce - telekomunikacja i internet

zakres=['402-300-01','402-300-02','402-300-03','402-300-04','402-300-05']
NAZWA='Usługi obce - telekomunikacyjne'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [400]:
# Usługi obce - badanie kruszyw

zakres=['402-400-02']
NAZWA='Usługi obce - badanie kruszyw'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
  1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=50

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [401]:
# Usługi obce - strzałowe SSE

zakres=['402-500-01']
NAZWA='Usługi obce - prace strzałowe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.15,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.25 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [402]:
# Usługi obce - pozostałe

zakres=['402-600-01','402-600-03','402-600-06','402-600-07','402-600-08','402-600-09','402-600-11','402-600-13','402-600-14','402-400-01']
NAZWA='Usługi obce - różne'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.15,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp


In [403]:
# Usługi obce - prawne

zakres=['402-600-05']
NAZWA='Usługi obce - prawne'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[t_zapisy['Konto'].isin(zakres)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1,
    1.05,
    1.05,
    1.05,
    1.05])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']<pd.to_datetime('2023-01-31') else -5000,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [404]:
# Usługi obce - dzierawy nowe złoze

zakres=['402-600-04']
NAZWA='Usługi obce - dzierżawa nowe złoże'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&t_zapisy['Opis'].str.contains('NOWE ZŁOŻE')].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2022-01-31') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [405]:
# Usługi obce - dzierawy stare złoze

zakres=['402-600-04']
NAZWA='Usługi obce - dzierżawa starego złoża'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&t_zapisy['Opis'].str.contains('STARE ZŁOŻE')].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:-x['OPEX_FIX'] if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [406]:
# Usługi obce - dzierawy stare złoze AMPLIZ

zakres=['402-600-04']
NAZWA='Usługi obce - dzierżawa starego złoża (Ampliz)'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&t_zapisy['Opis'].str.contains('WYROBISKA')].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1,
    1,
    1,
    1,
    1])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [407]:
# Usługi obce - dzierawy INNE



zakres=['402-600-04']
NAZWA='Usługi obce - pozostałe dzierżawy'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&(t_zapisy['Opis'].str.contains('WYROBISKA')==False)&(t_zapisy['Opis'].str.contains('NOWE ZŁOŻE')==False)&(t_zapisy['Opis'].str.contains('STARE ZŁOŻE')==False)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [408]:
# Usługi obce - POZOSTAŁE



zakres=['402-600-02']
NAZWA='Usługi obce - produkcja mieszanki'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&(t_zapisy['Opis'].str.contains('PRODUKCJA MIESZANKI'))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [409]:
# Usługi obce - doradcze



zakres=['402-600-02']
NAZWA='Usługi obce - doradztwo (B2B)'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&(t_zapisy['Opis'].str.contains('DORADZTW')|t_zapisy['Opis'].str.contains('DORADCZ'))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].iloc[7]
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [410]:
# Usługi obce - pozostałe



zakres=['402-600-02']
NAZWA='Usługi obce - pozostałe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))&(t_zapisy['Opis'].str.contains('DORADZTW')==False)&(t_zapisy['Opis'].str.contains('DORADCZ')==False)&(t_zapisy['Opis'].str.contains('PRODUKCJA MIESZANKI')==False)].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.9,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [411]:
# Usługi obce - bankowe


zakres=['402-700-01']
NAZWA='Usługi obce - bankowe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1,
    1,
    1,
    1,
    1])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [412]:
# Usługi obce - remontowo naprawcze


zakres=['402-800-01','402-800-02','402-800-03','402-800-04']
NAZWA='Usługi obce - naprawy i remonty'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.15,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.9,
    0.9,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.3 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [413]:
# Usługi obce - sprzętowe


zakres=['402-900-01','402-900-02']
NAZWA='Usługi obce - sprzętowe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.8,
    0.8,
    0.8,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:(x['OPEX_VAR']+x['OPEX_VAR'])*0.3 if (x['DATE']>=pd.to_datetime('2022-10-30') and x['DATE']<=pd.to_datetime('2022-12-31')) else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [414]:
# Usługi obce - księgowość

t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=t_ile
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    0.8,
    0.8,
    0.8,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')
temp['PPI']=temp.apply(lambda x:1 if x['DATE']<=pd.to_datetime('2022-09-30') else x['PPI'],axis=1)

#----------Zmienność kosztów -------------------------------------------------------

temp['Usługi obce - księgowość']=temp.apply(lambda x:-20000*5 if x['DATE']==pd.to_datetime('2022-09-30') else 0,axis=1)
temp['Usługi obce - księgowość']=temp.apply(lambda x:x['PPI']*-19000 if x['DATE']>pd.to_datetime('2022-09-30') else x['Usługi obce - księgowość'],axis=1)
temp_model=temp[['DATE','Usługi obce - księgowość']].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [415]:
# Leasing operacyjny

zakres=['402-600-10']

temp=t_balance[t_balance['Konto'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Usługi obce - leasing operacyjny'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [416]:
# Raport

temp=t_model[['R','Usługi obce - obsługa złoża','Usługi obce - transport mieszanki na plac','Usługi obce - telekomunikacyjne','Usługi obce - badanie kruszyw','Usługi obce - prace strzałowe','Usługi obce - pozostałe','Usługi obce - prawne','Usługi obce - dzierżawa nowe złoże','Usługi obce - dzierżawa starego złoża','Usługi obce - dzierżawa starego złoża (Ampliz)','Usługi obce - pozostałe dzierżawy','Usługi obce - produkcja mieszanki','Usługi obce - doradztwo (B2B)','Usługi obce - bankowe','Usługi obce - naprawy i remonty','Usługi obce - sprzętowe','Usługi obce - różne','Usługi obce - leasing operacyjny','Usługi obce - księgowość']].copy()

temp=temp.groupby(['R'],as_index=False)[['Usługi obce - obsługa złoża','Usługi obce - transport mieszanki na plac','Usługi obce - telekomunikacyjne','Usługi obce - badanie kruszyw','Usługi obce - prace strzałowe','Usługi obce - pozostałe','Usługi obce - prawne','Usługi obce - dzierżawa nowe złoże','Usługi obce - dzierżawa starego złoża','Usługi obce - dzierżawa starego złoża (Ampliz)','Usługi obce - pozostałe dzierżawy','Usługi obce - produkcja mieszanki','Usługi obce - doradztwo (B2B)','Usługi obce - bankowe','Usługi obce - naprawy i remonty','Usługi obce - sprzętowe','Usługi obce - różne','Usługi obce - leasing operacyjny','Usługi obce - księgowość']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '13_Usługi obce.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Usługi obce - obsługa złoża,"-4,598.31","-3,343.68","-3,960.35","-4,454.78","-4,869.98","-5,303.15"
Usługi obce - transport mieszanki na plac,-416.81,-39.13,-103.24,-141.50,-160.94,-168.80
Usługi obce - telekomunikacyjne,-108.64,-117.33,-126.72,-133.05,-136.38,-139.79
Usługi obce - badanie kruszyw,-54.88,-58.21,-63.04,-66.38,-68.24,-70.14
Usługi obce - prace strzałowe,"-2,216.86","-1,720.07","-1,956.23","-2,157.52","-2,317.52","-2,484.18"
Usługi obce - pozostałe,-819.15,-796.22,-859.91,-902.91,-925.48,-948.62
Usługi obce - prawne,-13.00,-73.00,-73.65,-74.34,-75.05,-75.81
Usługi obce - dzierżawa nowe złoże,-781.81,-844.36,-911.91,-957.50,-981.44,"-1,005.98"
Usługi obce - dzierżawa starego złoża,-158.45,-78.70,-26.37,-35.61,-40.46,-45.43
Usługi obce - dzierżawa starego złoża (Ampliz),-60.00,-60.00,-60.00,-60.00,-60.00,-60.00


#### Podatki i opłaty

In [417]:
# Podatki i opłaty - pozostałe


zakres=['403-001-01','403-001-02','403-002-02','403-003','403-004-01','403-004-02','403-005-01','403-005-02','403-006','403-008','403-010','403-007']
NAZWA='Podatki i opłaty - różne'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.025,
    1.025,
    1.025,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [418]:
# Podatki i opłaty - opłata eksploatacyjna


zakres=['403-011-01','403-011-02']
NAZWA='Podatki i opłaty - opłata eksploatacyjna'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [419]:
# Podatki i opłaty - grunty leśne


zakres=['403-012','403-015']
NAZWA='Podatki i opłaty - Lasy Państwowe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
   1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [420]:
# Raport

temp=t_model[['R','Podatki i opłaty - różne','Podatki i opłaty - opłata eksploatacyjna','Podatki i opłaty - Lasy Państwowe','Podatki i opłaty (planowane) - Lasy Państwowe']].copy()

temp=temp.groupby(['R'],as_index=False)[['Podatki i opłaty - różne','Podatki i opłaty - opłata eksploatacyjna','Podatki i opłaty - Lasy Państwowe','Podatki i opłaty (planowane) - Lasy Państwowe']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '14_Podatki i opłaty.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Podatki i opłaty - różne,-471.80,-483.59,-495.68,-508.07,-520.77,-533.79
Podatki i opłaty - opłata eksploatacyjna,"-1,461.93","-1,177.94","-1,339.83","-1,477.87","-1,587.63","-1,701.96"
Podatki i opłaty - Lasy Państwowe,-539.34,-582.48,-629.08,-660.54,-677.05,-693.98
Podatki i opłaty (planowane) - Lasy Państwowe,0.00,0.00,-127.80,-298.20,-340.80,-340.80
RAZEM,"-2,473.06","-2,244.01","-2,592.40","-2,944.68","-3,126.26","-3,270.53"


### Wynagrodzenia

In [421]:
# Wynagrodzenia


zakres=['R9']
NAZWA='Wynagrodzenia'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['ID'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.1,
    1.08,
    1.08,
    1.08,
    1.08])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1.05,
    1.05,
    1.05])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=-280000
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:-100000 if (x['DATE']>=pd.to_datetime('2022-01-31') and x['DATE'].month==12) else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:-100000 if (x['DATE']>=pd.to_datetime('2023-01-31') and x['DATE'].month==3) else x['OPEX_KOR'],axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:-(22500+20000)*3*x['PPI'] if (x['DATE']>=pd.to_datetime('2021-12-31') and x['DATE'].month==12) else x['OPEX_KOR'],axis=1) # Rezerwa na wynagrodzenie bonusowe Zarządu
temp['OPEX_KOR']=temp.apply(lambda x:-(7500)*3*x['PPI'] if (x['DATE']>=pd.to_datetime('2021-12-31') and x['DATE'].month==12) else x['OPEX_KOR'],axis=1) # Rezerwa na wynagrodzenie bonusowe RN
temp['OPEX_KOR']=temp.apply(lambda x:-50000 if (x['DATE']==pd.to_datetime('2022-10-31')) else x['OPEX_KOR'],axis=1)

temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [422]:
# ZUS


zakres=['R10']
NAZWA='Narzuty - ZUS'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['ID'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.1,
    1.08,
    1.08,
    1.08,
    1.08])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1.05,
    1.05,
    1.05])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].iloc[8]*270/320
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:-100000*0.2 if (x['DATE']>=pd.to_datetime('2022-01-31') and x['DATE'].month==12) else 0,axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:-100000*0.2 if (x['DATE']>=pd.to_datetime('2023-01-31') and x['DATE'].month==3) else x['OPEX_KOR'],axis=1)
temp['OPEX_KOR']=temp.apply(lambda x:-50000*0.2 if (x['DATE']==pd.to_datetime('2022-10-31')) else x['OPEX_KOR'],axis=1)

temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [423]:
# Raport

temp=t_model[['R','Wynagrodzenia','Narzuty - ZUS']].copy()

temp=temp.groupby(['R'],as_index=False)[['Wynagrodzenia','Narzuty - ZUS']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_Wynagrodzenia_ZUS.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Wynagrodzenia,"-3,541.20","-3,820.75","-4,118.41","-4,656.88","-5,267.50","-5,959.94"
Narzuty - ZUS,-888.63,"-1,062.30","-1,144.09","-1,292.03","-1,459.81","-1,650.06"
RAZEM,"-4,429.83","-4,883.05","-5,262.50","-5,948.91","-6,727.31","-7,610.00"


#### Pozostałe koszty operacyjne

In [424]:
# Pozostałe koszty operacyjne - podróe słubowe, samochody, reprezentacja

zakres=['409-0001','409-001','409-002','409-004','409-005','409-006-01','409-006-02','409-006-03']
NAZWA='Pozostałe koszty operacyjne - podróże i samochody'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1.2,
    1.1,
    1.1,
    1.1,
    1.1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [425]:
# Pozostałe koszty operacyjne - ubezpieczenia

zakres=['409-003-01','409-003-02','409-003-022','409-003-03','409-007']
NAZWA='Pozostałe koszty operacyjne - ubezpieczenia'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [426]:
# Pozostałe koszty operacyjne - Fundusz Likwidacji

zakres=['409-008-01','409-008-02']
NAZWA='Pozostałe koszty operacyjne - Fundusz Likwidacji'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
 1,
    1.08,
    1.08,
    1.05,
    1.025,
    1.025])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=0

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [427]:
# Pozostałe koszty operacyjne - rozliczenie projektów
# Tylko to traktujemy jak gotówkę

zakres=['409-009','409-010']
NAZWA='Pozostałe koszty operacyjne - projekty rozwojowe'

#----------Pobieranie danych-------------------------------------------

temp_zap=t_zapisy[(t_zapisy['Konto'].isin(zakres))].copy()
temp=temp_zap
temp['DATE']=pd.to_datetime(temp['Data'])+MonthEnd(0)
temp=temp.groupby(['DATE'],as_index=False)[['Kwota_SAL']].sum()
temp.rename(columns={'Kwota_SAL':'OPEX'},inplace=True)


t_ile=t_przychody[['DATE','PL_Ile_P']].copy()
temp=pd.merge(t_ile,temp,how='left',on='DATE')
temp.fillna(0,inplace=True)
temp['R']=temp['DATE'].dt.year

#------------ Wzrost cen i oszczędności----------------------------------------------

w_PPI=np.array([ # planowana inflacja
    1,
    1,
    1,
    1,
    1,
    1])
w_SAV=np.array([ # planowane oszczędności w latach
    1,
    1,
    1,
    1,
    1,
    1])

#-----------------------------------------------------------------------------------

w_PPI=w_PPI*w_SAV
w_PPI=np.cumprod(w_PPI).reshape(6,1)
w_lata=np.arange(2022,2028,1,dtype=int).reshape(6,1)
w_PPI=np.concatenate([w_lata,w_PPI],axis=1)
t_PPI=pd.DataFrame(w_PPI,columns=['R','PPI'])
t_PPI['R']=t_PPI['R'].astype(int)
temp=pd.merge(temp,t_PPI,how='left',on='R')

#----------Zmienność kosztów -------------------------------------------------------

K_STAŁE_PRC=100

K_STAŁE_M=temp['OPEX'].sum()/9
temp['OPEX_FIX']=temp.apply(lambda x:x['OPEX']*K_STAŁE_PRC/100 if x['DATE']<=pd.to_datetime('2022-09-30') else K_STAŁE_M*K_STAŁE_PRC/100,axis=1)
temp['OPEX_VAR']=temp['OPEX']*(1-K_STAŁE_PRC/100)
t_reg=temp[['PL_Ile_P','OPEX_VAR']].copy()
t_reg=t_reg.iloc[0:9]
x=t_reg['PL_Ile_P'].to_numpy()
y=t_reg['OPEX_VAR'].to_numpy()
x=x.reshape(-1,1)
model=LinearRegression()
model.fit(x,y)
A=model.coef_[0]
B=model.intercept_
temp['OPEX_VAR']=temp.apply(lambda x:x['OPEX_VAR'] if x['DATE']<=pd.to_datetime('2022-09-30') else A*x['PL_Ile_P']+B,axis=1)

#-----------Korekta kwotowa---------------------------------------------------------
temp['OPEX_KOR']=temp.apply(lambda x:0 if x['DATE']>=pd.to_datetime('2023-06-30') else 0,axis=1)
temp[NAZWA]=(temp['OPEX_FIX']+temp['OPEX_VAR'])*temp['PPI']+temp['OPEX_KOR']
temp_model=temp[['DATE',NAZWA]].copy()

#-----------Aktualizacja modelu ----------------------------------------------------

t_model=pd.merge(t_model,temp_model,how='left',on=['DATE'])
#temp_zap
#temp

In [428]:
# Raport

temp=t_model[['R','Pozostałe koszty operacyjne - podróże i samochody','Pozostałe koszty operacyjne - ubezpieczenia','Pozostałe koszty operacyjne - Fundusz Likwidacji','Pozostałe koszty operacyjne - projekty rozwojowe','Koszty zrealizowanych projektów (istniejące)','Koszty zrealizowanych projektów (planowane)']].copy()

temp=temp.groupby(['R'],as_index=False)[['Pozostałe koszty operacyjne - podróże i samochody','Pozostałe koszty operacyjne - ubezpieczenia','Pozostałe koszty operacyjne - Fundusz Likwidacji','Pozostałe koszty operacyjne - projekty rozwojowe','Koszty zrealizowanych projektów (istniejące)','Koszty zrealizowanych projektów (planowane)']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '15_Pozostałe_Koszty.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Pozostałe koszty operacyjne - podróże i samochody,-114.62,-148.55,-176.47,-203.83,-229.81,-259.11
Pozostałe koszty operacyjne - ubezpieczenia,-224.50,-242.46,-261.86,-274.95,-281.83,-288.87
Pozostałe koszty operacyjne - Fundusz Likwidacji,-146.19,-117.79,-133.98,-147.79,-158.76,-170.20
Pozostałe koszty operacyjne - projekty rozwojowe,-163.50,-163.50,-163.50,-163.50,-163.50,-163.50
Koszty zrealizowanych projektów (istniejące),-383.78,-459.26,-459.26,-459.26,-459.26,-459.26
Koszty zrealizowanych projektów (planowane),0.00,0.00,-45.57,-114.05,-131.81,-131.81
RAZEM,"-1,032.59","-1,131.56","-1,240.64","-1,363.37","-1,424.97","-1,472.75"


In [429]:
# Koszty - wartość sprzedanych TiM

zakres=['R12']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Wartość sprzedanych towarów i materiałów'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

#### Pozostałe przychody i koszty

In [430]:
# PPO

zakres=['R13','R14']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'POZOSTAŁE PRZYCHODY OPERACYJNE'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)


In [431]:
# PKO

zakres=['R15']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'POZOSTAŁE KOSZTY OPERACYJNE'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

#### Przychody i koszty finansowe

In [432]:
# P_Fin

zakres=['R16']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Przychody finansowe'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [433]:
# K_Fin

zakres=['R17','R18']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Koszty finansowe (bieżące)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

KFIN=t_model['Koszty finansowe (bieżące)'].iloc[8]
t_model['Koszty finansowe (bieżące)']=t_model.apply(lambda x:x['Koszty finansowe (bieżące)'] if x['DATE']<=pd.to_datetime('2022-09-30') else KFIN,axis=1)

#### Podatek dochodowy

In [434]:
# Podatek_dochodowy

zakres=['R19']

temp=t_balance[t_balance['ID'].isin(zakres)]
temp=temp.groupby(['M'],as_index=False)[['Kwota_SAL']].sum()
temp['Kwota_SAL']=temp['Kwota_SAL']
temp.rename(columns={'Kwota_SAL':'Podatek dochodowy (realizacja)'},inplace=True)
temp['R']=2022
temp=temp[temp['M']<=9]
t_model=pd.merge(t_model,temp,how='left',on=['M','R'])
t_model.fillna(0,inplace=True)

In [435]:
# Raport

temp=t_model[['R','Przychody finansowe','Koszty finansowe (bieżące)','Koszty finansowe (inwestycje)']].copy()

temp=temp.groupby(['R'],as_index=False)[['Przychody finansowe','Koszty finansowe (bieżące)','Koszty finansowe (inwestycje)']].sum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela.loc['RAZEM']=tabela.sum(numeric_only=True,axis=0)
tabela.loc['RAZEM','POZYCJA']='RAZEM'
last_row = pd.IndexSlice[tabela.index[tabela.index == "RAZEM"], :]

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '16_Wynik_Finansowy.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Przychody finansowe,0.00,0.00,0.00,0.00,0.00,0.00
Koszty finansowe (bieżące),-216.78,-238.83,-238.83,-238.83,-238.83,-238.83
Koszty finansowe (inwestycje),0.00,-17.46,-30.92,-25.22,-19.03,-12.34
RAZEM,-216.78,-256.29,-269.75,-264.04,-257.86,-251.17


## Mapowanie sprawozdań

#### Rachunek Wyników

In [436]:
t_model['A. PRZYCHODY NETTO']=t_model[[
    'Przychody pozostałe',
    'Przychody ze sprzedaży produktów (planowane)',
    'Przychody ze sprzedaży produktów (zrealizowane)',
    'Koszty bezpośrednie sprzedaży (transport)']].sum(axis=1)

t_model['1. Amortyzacja']=t_model[[
    'Amortyzacja (inwestycje)',
    'Amortyzacja (istniejące)']].sum(axis=1)

t_model['2. Zużycie materiałów i energii']=t_model[[
    'Zuzycie materiałów pozostałych', 
    'Zużycie energii',
    'Zużycie materiałów produkcyjnych',
    'Zużycie paliwa']].sum(axis=1)

t_model['3. Usługi obce']=t_model[[
    'Usługi obce - badanie kruszyw', 
    'Usługi obce - bankowe',
    'Usługi obce - doradztwo (B2B)',
    'Usługi obce - dzierżawa nowe złoże',
    'Usługi obce - dzierżawa starego złoża',
    'Usługi obce - dzierżawa starego złoża (Ampliz)',
    'Usługi obce - naprawy i remonty', 
    'Usługi obce - obsługa złoża',
    'Usługi obce - pozostałe', 'Usługi obce - pozostałe dzierżawy',
    'Usługi obce - prace strzałowe', 'Usługi obce - prawne',
    'Usługi obce - produkcja mieszanki', 'Usługi obce - różne',
    'Usługi obce - sprzętowe',
    'Usługi obce - transport mieszanki na plac',
    'Usługi obce - telekomunikacyjne',
    'Usługi obce - leasing operacyjny',
    'Usługi obce - księgowość']].sum(axis=1)

t_model['4. Podatki i opłaty']=t_model[[
    'Podatki i opłaty (planowane) - Lasy Państwowe',
    'Podatki i opłaty - Lasy Państwowe',
    'Podatki i opłaty - opłata eksploatacyjna',
    'Podatki i opłaty - różne']].sum(axis=1)

t_model['5. Wynagrodzenia']=t_model[['Wynagrodzenia']].sum(axis=1)
t_model['6. Ubezpieczenia społeczne']=t_model[['Narzuty - ZUS']].sum(axis=1)

t_model['7. Pozostałe koszty rodzajowe']=t_model[[
    'Pozostałe koszty operacyjne - Fundusz Likwidacji',
    'Pozostałe koszty operacyjne - podróże i samochody',
    'Pozostałe koszty operacyjne - projekty rozwojowe',
    'Koszty zrealizowanych projektów (planowane)',
    'Pozostałe koszty operacyjne - ubezpieczenia']].sum(axis=1)

t_model['8. Wartość sprzedanych towarów i materiałów']=t_model[['Wartość sprzedanych towarów i materiałów']].sum(axis=1)

t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']=t_model[[ 
    '1. Amortyzacja',
    '2. Zużycie materiałów i energii', 
    '3. Usługi obce',
    '4. Podatki i opłaty', 
    '5. Wynagrodzenia',
    '6. Ubezpieczenia społeczne', 
    '7. Pozostałe koszty rodzajowe',
    '8. Wartość sprzedanych towarów i materiałów']].sum(axis=1)

t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']=t_model[['POZOSTAŁE PRZYCHODY OPERACYJNE']].sum(axis=1)
t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']=t_model[['POZOSTAŁE KOSZTY OPERACYJNE']].sum(axis=1)
t_model['E. PRZYCHODY FINANSOWE']=t_model[['Przychody finansowe']].sum(axis=1)
t_model['F. KOSZTY FINANSOWE']=t_model[[
    'Koszty finansowe (bieżące)', 
    'Koszty finansowe (inwestycje)']].sum(axis=1)

t_model['G. WYNIK BRUTTO']=t_model[[
    'A. PRZYCHODY NETTO',
    'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
    'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
    'D. POZOSTAŁE KOSZTY OPERACYJNE',
    'E. PRZYCHODY FINANSOWE',
    'F. KOSZTY FINANSOWE']].sum(axis=1)

t_model['Podatek (planowany)']=t_model.apply(lambda x:-x['G. WYNIK BRUTTO']*0.19 if x['DATE']>=pd.to_datetime('2022-09-30') else 0,axis=1)

t_model['H. PODATEK DOCHODOWY']=t_model[[
    'Podatek dochodowy (realizacja)',
    'Podatek (planowany)']].sum(axis=1)

t_model['I. WYNIK NETTO']=t_model[[
    'G. WYNIK BRUTTO',
    'H. PODATEK DOCHODOWY']].sum(axis=1)

### Rozliczenia bilansowe

In [437]:

t_model['Aktywa_CF']=t_model[[
    'Wartości niematerialne i prawne',
    'Rzeczowe aktywa trwałe (istniejące)',
    'Rzeczowe aktywa trwałe (inwestycje)',
    'Długoterminowe rozliczenia',
    'Długoterminowe rozliczenia (planowane)',
    'Zapasy',
    'Należności',
    'Środki pieniężne']].sum(axis=1)
t_model['Pasywa_CF']=t_model[[
    'Kapitał podstawowy',
    'Kapitał zapasowy',
    'Kapitał z aktualizacji wyceny',
    'Pozostałe kapitały rezerwowe',
    'Zysk (strata) z lat ubiegłych',
    'I. WYNIK NETTO',
    'Odpisy w ciągu roku',
    'Rezerwy na zobowiązania',
    'Zobowiązania długoterminowe (PEKAO)',
    'Zobowiązania długoterminowe (Tarcza)',
    'Zobowiązania długoterminowe (inwestycje)', 
    'Zobowiązania długoterminowe (leasing)',
    'Pożyczka WFOŚiGW',
    'Zobowiązania krótkoterminowe (dywidenda)',
    'Zobowiązania krótkoterminowe (podatki i wynagrodzenia)',
    'Zobowiązania krótkoterminowe (pozostałe)',
    'Zobowiązania z tytułu dostaw i usług',
    'Kredyt obrotowy (PEKAO)',
    'Rozliczenia międzyokresowe przychodów',
    'Fundusz Socjalny', 
    'Fundusz Likwidacji Zakładu Górniczego']].sum(axis=1)

t_model['Zapotrzebowanie na CF']=t_model['Pasywa_CF']-t_model['Aktywa_CF']

In [438]:
#t_model['Zapotrzebowanie na CF']=0
#t_model[['DATE','Zapotrzebowanie na CF','Aktywa_CF','Pasywa_CF']].head(12)

In [439]:
# Mapowanie bilansu - Aktywa

t_model['1. Wartości niematerialne i prawne']=t_model[[
    'Wartości niematerialne i prawne']].sum(axis=1)

t_model['2. Rzeczowe aktywa trwałe']=t_model[[
    'Rzeczowe aktywa trwałe (istniejące)',
    'Rzeczowe aktywa trwałe (inwestycje)']].sum(axis=1)

t_model['3. Należności długoterminowe']=0.0

t_model['4. Długoterminowe rozliczenia międzyokresowe']=t_model[[
    'Długoterminowe rozliczenia',
    'Długoterminowe rozliczenia (planowane)']].sum(axis=1)

t_model['A. AKTYWA TRWAŁE']=t_model[[ 
    '1. Wartości niematerialne i prawne',
    '2. Rzeczowe aktywa trwałe',
    '3. Należności długoterminowe',
    '4. Długoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['1. Zapasy']=t_model[['Zapasy']].sum(axis=1)

t_model['2. Należności krótkoterminowe']=t_model[['Należności']].sum(axis=1)

t_model['3. Inwestycje krótkoterminowe']=t_model[['Środki pieniężne','Zapotrzebowanie na CF']].sum(axis=1)

t_model['4. Krótkoterminowe rozliczenia międzyokresowe']=0.0

t_model['B. AKTYWA OBROTOWE']=t_model[[ 
    '1. Zapasy',
    '2. Należności krótkoterminowe',
    '3. Inwestycje krótkoterminowe',
    '4. Krótkoterminowe rozliczenia międzyokresowe']].sum(axis=1)

t_model['AKTYWA RAZEM']=t_model[['A. AKTYWA TRWAŁE','B. AKTYWA OBROTOWE']].sum(axis=1)



In [440]:
# Mapowanie bilansu - Pasywa

t_model['1. Kapitał podstawowy']=t_model[[
    'Kapitał podstawowy']].sum(axis=1)

t_model['2. Kapitał zapasowy']=t_model[[
    'Kapitał zapasowy']].sum(axis=1)

t_model['3. Kapitał z aktualizacji wyceny']=t_model[[
    'Kapitał z aktualizacji wyceny']].sum(axis=1)

t_model['4. Pozostałe kapitały rezerwowe']=t_model[[
    'Pozostałe kapitały rezerwowe']].sum(axis=1)

t_model['5. Zysk (strata) z lat ubiegłych']=t_model[[
    'Zysk (strata) z lat ubiegłych']].sum(axis=1)

t_model['6. Zysk (strata)']=t_model['I. WYNIK NETTO']

t_model['7. Odpisy w ciągu roku']=t_model[[
    'Odpisy w ciągu roku']].sum(axis=1)

t_model['A. KAPITAŁ WŁASNY']=t_model[[ 
    '1. Kapitał podstawowy',
    '2. Kapitał zapasowy',
    '3. Kapitał z aktualizacji wyceny',
    '4. Pozostałe kapitały rezerwowe',
    '5. Zysk (strata) z lat ubiegłych',
    '6. Zysk (strata)',
    '7. Odpisy w ciągu roku']].sum(axis=1)

t_model['1. Rezerwy na zobowiązania']=t_model[['Rezerwy na zobowiązania']].sum(axis=1)

t_model['2. Zobowiązania długoterminowe']=t_model[[
    'Zobowiązania długoterminowe (PEKAO)',
    'Zobowiązania długoterminowe (Tarcza)',
    'Zobowiązania długoterminowe (inwestycje)',
    'Zobowiązania długoterminowe (leasing)',
    'Pożyczka WFOŚiGW']].sum(axis=1)

t_model['3. Zobowiązania krótkoterminowe']=t_model[[
    'Zobowiązania krótkoterminowe (dywidenda)',
    'Zobowiązania krótkoterminowe (podatki i wynagrodzenia)',
    'Zobowiązania krótkoterminowe (pozostałe)',
    'Zobowiązania z tytułu dostaw i usług',
    'Kredyt obrotowy (PEKAO)']].sum(axis=1)

t_model['4. Rozliczenia międzyokresowe']=t_model[[
     'Rozliczenia międzyokresowe przychodów']].sum(axis=1)

t_model['5. Fundusze specjalne']=t_model[[
     'Fundusz Socjalny',
     'Fundusz Likwidacji Zakładu Górniczego']].sum(axis=1)

t_model['B. ZOBOWIĄZANIA I REZERWY']=t_model[[ 
    '1. Rezerwy na zobowiązania',
    '2. Zobowiązania długoterminowe',
    '3. Zobowiązania krótkoterminowe',
    '4. Rozliczenia międzyokresowe',
    '5. Fundusze specjalne']].sum(axis=1)


t_model['PASYWA RAZEM']=t_model[['A. KAPITAŁ WŁASNY', 'B. ZOBOWIĄZANIA I REZERWY']].sum(axis=1)

### Raport miesięczny dla roku

In [441]:
# Raport miesięczny dla roku

ROK=2022


temp=t_model[[
       'DATE',
       'M',
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].copy()

temp=temp[temp['DATE'].dt.year==ROK].copy()
temp['Q']=pd.PeriodIndex(temp['DATE'],freq='Q')


#temp['Q']=temp['Q'].astype(str)
temp=temp.groupby(['Q'],as_index=False)[['A. PRZYCHODY NETTO','B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ', '1. Amortyzacja','2. Zużycie materiałów i energii','3. Usługi obce','4. Podatki i opłaty','5. Wynagrodzenia', '6. Ubezpieczenia społeczne','7. Pozostałe koszty rodzajowe','8. Wartość sprzedanych towarów i materiałów', 'C. POZOSTAŁE PRZYCHODY OPERACYJNE','D. POZOSTAŁE KOSZTY OPERACYJNE','E. PRZYCHODY FINANSOWE',       'F. KOSZTY FINANSOWE', 'G. WYNIK BRUTTO','H. PODATEK DOCHODOWY','I. WYNIK NETTO']].sum()



temp.rename(columns={'Q':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4'])
tabela=temp[['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4']].copy()
tabela['RAZEM']=tabela[['2022Q1', '2022Q2', '2022Q3','2022Q4']].sum(axis=1)


tabela=tabela.style.format({'2022Q1':'{0:,.2f}','2022Q2':'{0:,.2f}','2022Q3':'{0:,.2f}','2022Q4':'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '17_Wynik_2022.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022Q1,2022Q2,2022Q3,2022Q4,RAZEM
A. PRZYCHODY NETTO,"5,490.28","8,579.83","12,421.58","8,338.91","34,830.59"
B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ,"-6,083.47","-7,700.31","-9,097.10","-7,762.21","-30,643.10"
1. Amortyzacja,-388.53,-390.24,-395.69,-388.61,"-1,563.06"
2. Zużycie materiałów i energii,"-1,237.18","-1,619.35","-2,060.31","-1,826.78","-6,743.62"
3. Usługi obce,"-2,769.30","-3,771.04","-4,080.12","-3,651.24","-14,271.71"
4. Podatki i opłaty,-549.95,-616.50,-742.64,-563.97,"-2,473.06"
5. Wynagrodzenia,-715.23,-870.09,"-1,043.38",-912.50,"-3,541.20"
6. Ubezpieczenia społeczne,-190.20,-191.74,-244.35,-262.34,-888.63
7. Pozostałe koszty rodzajowe,-229.77,-146.97,-115.29,-156.77,-648.81
8. Wartość sprzedanych towarów i materiałów,-3.31,-94.37,-415.32,0.00,-513.00


In [442]:
temp=t_model[[
       'DATE',
       'M',
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].copy()

temp=temp[temp['DATE'].dt.year==ROK].copy()
temp['Q']=pd.PeriodIndex(temp['DATE'],freq='Q')


#temp['Q']=temp['Q'].astype(str)
temp=temp.groupby(['Q'],as_index=False)[['A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].sum()



temp.rename(columns={'Q':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4'])
tabela=temp[['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4']].copy()
tabela['RAZEM']=tabela[['2022Q1', '2022Q2', '2022Q3','2022Q4']].sum(axis=1)


tabela=tabela.style.format({'2022Q1':'{0:,.2f}','2022Q2':'{0:,.2f}','2022Q3':'{0:,.2f}','2022Q4':'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '18_Aktywa_2022.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022Q1,2022Q2,2022Q3,2022Q4,RAZEM
A. AKTYWA TRWAŁE,"17,668.10",585.12,240.35,-503.42,"17,990.15"
1. Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00
2. Rzeczowe aktywa trwałe,"13,088.91",-299.24,-153.90,-388.61,"12,247.17"
3. Należności długoterminowe,0.00,0.00,0.00,0.00,0.00
4. Długoterminowe rozliczenia międzyokresowe,"4,579.19",884.35,394.25,-114.81,"5,742.99"
B. AKTYWA OBROTOWE,"9,663.84",48.13,"2,529.00",-978.21,"11,262.76"
1. Zapasy,"1,658.11",-571.37,-208.36,0.00,878.37
2. Należności krótkoterminowe,"6,506.98",557.39,"3,538.76",0.00,"10,603.13"
3. Inwestycje krótkoterminowe,"1,498.76",62.11,-801.39,-978.21,-218.74
4. Krótkoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00


In [443]:
temp=t_model[[
       'DATE',
       'M',
       'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].copy()

temp=temp[temp['DATE'].dt.year==ROK].copy()
temp['Q']=pd.PeriodIndex(temp['DATE'],freq='Q')


#temp['Q']=temp['Q'].astype(str)
temp=temp.groupby(['Q'],as_index=False)[['A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].sum()



temp.rename(columns={'Q':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4'])
tabela=temp[['POZYCJA','2022Q1', '2022Q2', '2022Q3','2022Q4']].copy()
tabela['RAZEM']=tabela[['2022Q1', '2022Q2', '2022Q3','2022Q4']].sum(axis=1)


tabela=tabela.style.format({'2022Q1':'{0:,.2f}','2022Q2':'{0:,.2f}','2022Q3':'{0:,.2f}','2022Q4':'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '19_Pasywa_2022.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')
tabela

POZYCJA,2022Q1,2022Q2,2022Q3,2022Q4,RAZEM
A. KAPITAŁ WŁASNY,"16,079.90",787.81,-9.63,418.76,"17,276.85"
1. Kapitał podstawowy,"1,240.00",0.00,0.00,0.00,"1,240.00"
2. Kapitał zapasowy,"9,561.29",0.00,"1,422.31",0.00,"10,983.60"
3. Kapitał z aktualizacji wyceny,74.06,0.00,0.00,0.00,74.06
4. Pozostałe kapitały rezerwowe,"4,653.93",0.00,0.00,0.00,"4,653.93"
5. Zysk (strata) z lat ubiegłych,"4,422.31",0.00,"-4,422.31",0.00,0.00
6. Zysk (strata),-871.70,787.81,"2,990.37",418.76,"3,325.25"
7. Odpisy w ciągu roku,"-3,000.00",0.00,0.00,0.00,"-3,000.00"
B. ZOBOWIĄZANIA I REZERWY,"11,252.04",-154.57,"2,778.98","-1,900.39","11,976.06"
1. Rezerwy na zobowiązania,"2,313.33",-187.65,-76.37,0.00,"2,049.32"


### RAPORT ROCZNY

In [444]:
# Raport roczny
temp=t_model[[
       'R',
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. PRZYCHODY NETTO',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ',
       '1. Amortyzacja',
       '2. Zużycie materiałów i energii',
       '3. Usługi obce',
       '4. Podatki i opłaty',
       '5. Wynagrodzenia',
       '6. Ubezpieczenia społeczne',
       '7. Pozostałe koszty rodzajowe',
       '8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE',
       'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE',
       'F. KOSZTY FINANSOWE',
       'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY',
       'I. WYNIK NETTO']].sum()

t_pbt=temp[['R','G. WYNIK BRUTTO']].copy()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')

tabela.to_latex(ASSETS / '20_Rachunek_Wyników.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. PRZYCHODY NETTO,"34,830.59","30,369.84","32,590.38","36,186.51","39,534.16","42,588.18"
B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ,"-30,643.10","-27,535.00","-29,895.47","-32,493.71","-34,527.66","-36,530.28"
1. Amortyzacja,"-1,563.06","-1,593.40","-1,560.52","-1,466.21","-1,253.57","-1,016.45"
2. Zużycie materiałów i energii,"-6,743.62","-5,600.09","-5,960.14","-6,522.61","-6,929.06","-7,256.20"
3. Usługi obce,"-14,271.71","-12,542.15","-13,738.54","-14,707.19","-15,525.76","-16,363.60"
4. Podatki i opłaty,"-2,473.06","-2,244.01","-2,592.40","-2,944.68","-3,126.26","-3,270.53"
5. Wynagrodzenia,"-3,541.20","-3,820.75","-4,118.41","-4,656.88","-5,267.50","-5,959.94"
6. Ubezpieczenia społeczne,-888.63,"-1,062.30","-1,144.09","-1,292.03","-1,459.81","-1,650.06"
7. Pozostałe koszty rodzajowe,-648.81,-672.30,-781.38,-904.11,-965.71,"-1,013.49"
8. Wartość sprzedanych towarów i materiałów,-513.00,0.00,0.00,0.00,0.00,0.00


In [445]:
# Raport roczny
temp=t_model[[
       'R',
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. AKTYWA TRWAŁE',
       '1. Wartości niematerialne i prawne',
       '2. Rzeczowe aktywa trwałe',
       '3. Należności długoterminowe',
       '4. Długoterminowe rozliczenia międzyokresowe',
       'B. AKTYWA OBROTOWE',
       '1. Zapasy',
       '2. Należności krótkoterminowe',
       '3. Inwestycje krótkoterminowe',
       '4. Krótkoterminowe rozliczenia międzyokresowe',
       'AKTYWA RAZEM']].sum()

t_cf=temp[['R','3. Inwestycje krótkoterminowe']].copy()
temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '21_Aktywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. AKTYWA TRWAŁE,"17,990.15","16,537.49","16,072.15","15,832.63","13,987.99","12,380.47"
1. Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00,0.00
2. Rzeczowe aktywa trwałe,"12,247.17","11,253.77","9,693.25","8,227.03","6,973.47","5,957.01"
3. Należności długoterminowe,0.00,0.00,0.00,0.00,0.00,0.00
4. Długoterminowe rozliczenia międzyokresowe,"5,742.99","5,283.73","6,378.90","7,605.60","7,014.53","6,423.46"
B. AKTYWA OBROTOWE,"11,262.76","13,930.84","15,984.53","18,859.98","24,446.27","30,645.79"
1. Zapasy,878.37,878.37,878.37,878.37,878.37,878.37
2. Należności krótkoterminowe,"10,603.13","10,603.13","10,603.13","10,603.13","10,603.13","10,603.13"
3. Inwestycje krótkoterminowe,-218.74,"2,449.34","4,503.03","7,378.49","12,964.77","19,164.30"
4. Krótkoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00


In [446]:
# Raport roczny
temp=t_model[[
       'R',
      'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].copy()

temp=temp.groupby(['R'],as_index=False)[[
       'A. KAPITAŁ WŁASNY',
       '1. Kapitał podstawowy',
       '2. Kapitał zapasowy',
       '3. Kapitał z aktualizacji wyceny',
       '4. Pozostałe kapitały rezerwowe',
       '5. Zysk (strata) z lat ubiegłych',
       '6. Zysk (strata)',
       '7. Odpisy w ciągu roku',
       'B. ZOBOWIĄZANIA I REZERWY',
       '1. Rezerwy na zobowiązania',
       '2. Zobowiązania długoterminowe',
       '3. Zobowiązania krótkoterminowe',
       '4. Rozliczenia międzyokresowe',
       '5. Fundusze specjalne',
       'PASYWA RAZEM']].sum()

temp=temp.cumsum()
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '22_Pasywa.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela

POZYCJA,2022,2023,2024,2025,2026,2027
A. KAPITAŁ WŁASNY,"17,276.85","19,365.47","21,329.84","24,107.13","27,953.53","32,656.98"
1. Kapitał podstawowy,"1,240.00","1,240.00","1,240.00","1,240.00","1,240.00","1,240.00"
2. Kapitał zapasowy,"10,983.60","10,983.60","10,983.60","10,983.60","10,983.60","10,983.60"
3. Kapitał z aktualizacji wyceny,74.06,74.06,74.06,74.06,74.06,74.06
4. Pozostałe kapitały rezerwowe,"4,653.93","4,653.93","4,653.93","4,653.93","4,653.93","4,653.93"
5. Zysk (strata) z lat ubiegłych,0.00,0.00,0.00,0.00,0.00,0.00
6. Zysk (strata),"3,325.25","5,413.88","7,378.25","10,155.54","14,001.93","18,705.38"
7. Odpisy w ciągu roku,"-3,000.00","-3,000.00","-3,000.00","-3,000.00","-3,000.00","-3,000.00"
B. ZOBOWIĄZANIA I REZERWY,"11,976.06","11,102.86","10,726.83","10,585.48","10,480.73","10,369.29"
1. Rezerwy na zobowiązania,"2,049.32","2,049.32","2,049.32","2,049.32","2,049.32","2,049.32"


In [447]:
t_pbt_cf=pd.merge(t_pbt,t_cf,how='inner',on=['R'])
t_pbt_cf.rename(columns={'G. WYNIK BRUTTO':'Wynik (PBT)','3. Inwestycje krótkoterminowe':'Środki pieniężne'},inplace=True)

zm_saving=[[2023,1632.94*1000],[2024,2238.44*1000],[2025,1856.98*1000],[2026,1536.06*1000],[2027,1166.71*1000]]
t_zm_saving=pd.DataFrame(zm_saving,columns=['R','Oszczędności (Mobil)'])
t_pbt_cf=pd.merge(t_pbt_cf,t_zm_saving,how='left',on=['R'])
t_pbt_cf['Wynik (PBT) z ZM']=t_pbt_cf['Wynik (PBT)']+t_pbt_cf['Oszczędności (Mobil)']
t_pbt_cf['Środki pieniężne z ZM']=t_pbt_cf['Środki pieniężne']+t_pbt_cf['Oszczędności (Mobil)']

In [448]:
temp=t_pbt_cf
temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()

temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2022,2023,2024,2025,2026,2027])
tabela=temp[['POZYCJA',2022,2023,2024,2025,2026,2027]].copy()

tabela=tabela.style.format({2022:'{0:,.2f}',2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}',2027:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '23_Podsumowanie.tex',hrules=True,column_format='L{5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela

POZYCJA,2022,2023,2024,2025,2026,2027
Wynik (PBT),"3,772.85","2,578.55","2,425.15","3,428.75","4,748.64","5,806.73"
Środki pieniężne,-218.74,"2,668.08","2,053.69","2,875.46","5,586.28","6,199.53"
Oszczędności (Mobil),nan,"1,632.94","2,238.44","1,856.98","1,536.06","1,166.71"
Wynik (PBT) z ZM,nan,"4,211.49","4,663.59","5,285.73","6,284.70","6,973.44"
Środki pieniężne z ZM,nan,"4,301.02","4,292.13","4,732.44","7,122.34","7,366.24"
